## Calculate Precipitation-Buoyancy POD

This notebook calculates the precipitation-buoyancy relationship POD as defined in [Ahmed and Neelin (2021)](https://agupubs.onlinelibrary.wiley.com/doi/full/10.1029/2021GL094108).

## Import Necessary Packages

In [ ]:
import glob
import warnings
import numpy as np
import xarray as xr
from datetime import datetime
warnings.filterwarnings('ignore')

## User-Defined Fields

In [ ]:
AUTHOR   = 'Savannah L. Ferretti'
EMAIL    = 'savannah.ferretti@uci.edu'
FILEDIR  = '/ocean/projects/atm200007p/sferrett/Repos/monsoon-pr/data/raw'
SAVEDIR  = '/ocean/projects/atm200007p/sferrett/Repos/monsoon-pr/data/processed'

## Load Variables

Load in preprocessed precipitation (mm/day), surface pressure ($p_s$, hPa), temperature ($T$, K), and specific humidity ($q$, kg/kg) data. Create a 4D pressure ($p$, hPa) dataset from the levels that $T$ and $q$ are available on.

In [ ]:
def load_variable(filename,filedir=FILEDIR):
    filepath = f'{filedir}/{filename}.nc'
    data = xr.open_dataset(filepath)
    return data.load()

In [ ]:
pr = load_variable('IMERG_precipitation_flux.nc').pr
ps = load_variable('ERA5_surface_pressure.nc').ps
t  = load_variable('ERA5_temperature.nc').t
q  = load_variable('ERA5_specific_humidity.nc').q
p  = (t.lev).expand_dims({'time':t.time,'lat':t.lat,'lon':t.lon},(1,2,3))

## Calculate Equivalent Potential Temperature

```calc_thetae()``` calculates equivalent potential temperature ($\theta_e$, K) following [Bolton (1980)](https://journals.ametsoc.org/view/journals/mwre/108/7/1520-0493_1980_108_1046_tcoept_2_0_co_2.xml?tab_body=pdf) Equation 43 (where $T_L$ is given by Equation 55):
    
$$ \theta_e = T\left(\frac{1000}{p}\right)^{0.2854\left(1-0.28 \times 10^{-3}r\right)} \times \text{ exp}\left[\left(\frac{3.376}{T_L}-0.00254\right) \times r\left(1+0.81 \times 10^{-3}r\right)\right]$$

Saturated equivalent potential temperature ($\theta^*_e$) is calculated by substituting $q$ for its saturated counterpart ($q_s$). It is calculated following Equation 4 from [Plymouth State Weather Center (2018)](https://vortex.plymouth.edu/~stmiller/stmiller_content/Publications/AtmosRH_Equations_Rev.pdf), where saturation vapor pressure ($e_s$, hPa) is calculated following Equations 17 and 18 from [Kuang (2018)](https://journals.ametsoc.org/view/journals/apme/57/6/jamc-d-17-0334.1.xml)

In [ ]:
# def calc_es(t):
#     c0 = 0.6105851e+03
#     c1 = 0.4440316e+02
#     c2 = 0.1430341e+01
#     c3 = 0.2641412e-01
#     c4 = 0.2995057e-03
#     c5 = 0.2031998e-05
#     c6 = 0.6936113e-08
#     c7 = 0.2564861e-11
#     c8 = -.3704404e-13
#     tc = t-273.15
#     bolton = 6.112*np.exp(17.67*tc/(243.5+tc))
#     poly   = (c0+tc*(c1+tc*(c2+tc*(c3+tc*(c4+tc*(c5+tc*(c6+tc*(c7+tc*c8))))))))/100. # Pa to hPa
#     es = np.where(tc<-80.,bolton,poly)
#     return es

def calc_es(t):
    tc = t-273.15
    eswat = np.exp(34.494-(4924.99/(tc+237.1)))/((tc+105.)**1.57)
    esice = np.exp(43.494-(6545.8/(tc+278.)))/((tc+868.)**2)
    es = (np.where(tc>0.,esw,esi)) 
    es = es/100. # Pa to hPa
    return es

def calc_qs(p,t):
    rv = 461.50   
    rd = 287.04    
    es = calc_es(t) 
    epsilon = rd/rv
    qs = (epsilon*es)/(p-es*(1.-epsilon))
    return qs


def calc_thetae(p,t,q):
    p0 = 1000.  
    rv = 461.5  
    rd = 287.04
    epsilon = rd/rv
    r  = q/(1.-q) 
    e  = (t.lev*r)/(epsilon+r)
    tl = 2840./(3.5*np.log(t)-np.log(e)-4.805)+55.
    thetae = t*(p0/p)**(0.2854*(1.-0.28*r))*np.exp((3.376/tl-0.00254)*1000.*r*(1.+0.81*r))
    return thetae

In [ ]:
qs = calc_qs(p,t)
thetae  = calc_thetae(p,t,q)
thetaes = calc_thetae(p,t,qs)

## Calculate Layer Averages

The boundary layer is defined as a 100 hPa thick layer above the surface, and the lower free-troposphere is defined as the layer extending from the top of the boundary layer to 500 hPa.

In [ ]:
pbltop = ps-100.
lfttop = xr.full_like(ps,500.) 

Calculate the average $\theta_e$ over the boundary layer ($\theta_{eB}$) and lower free-troposphere ($\theta_{eL}$),  as well as ${\theta^*_e}$ over the lower-free troposphere (${\theta^*_{eL}}$). ```get_p_above()``` and ```get_p_below()``` finds the bounds of the specified atmospheric layer, and ```calc_layer_average()``` uses numerical integration to calculates the average within that layer.

In [ ]:
def get_p_above(a,levels,side):
    idx    = np.searchsorted(levels,a,side=side)
    pabove = levels[np.maximum(idx-1,0)]
    return pabove

def get_p_below(a,levels,side):
    idx    = np.searchsorted(levels,a,side=side)
    pbelow = levels[np.minimum(idx,len(levels)-1)]
    return pbelow

def calc_layer_average(data,a,b):
    a,b,data = a.load(),b.load(),data.load()
    pabove = xr.apply_ufunc(get_p_above,a,kwargs={'levels':np.array(data.lev),'side':'right'})
    pbelow = xr.apply_ufunc(get_p_below,a,kwargs={'levels':np.array(data.lev),'side':'right'})
    fabove = data.sel({'lev':pabove})
    fbelow = data.sel({'lev':pbelow})
    correction = -fabove/2*(pbelow-pabove)*(a<data.lev[-1])
    pbelow += (pbelow==pabove) 
    belowintegral = (a-pabove)*fabove+(fbelow-fabove)*(a-pabove)**2/(pbelow-pabove)/2+correction
    innerintegral = (data*(data.lev<=a)*(data.lev>=b)).integrate('lev')
    pabove = xr.apply_ufunc(get_p_above,b,kwargs={'levels':np.array(data.lev),'side':'left'})
    pbelow = xr.apply_ufunc(get_p_below,b,kwargs={'levels':np.array(data.lev),'side':'left'})
    fabove = data.sel({'lev':pabove})
    fbelow = data.sel({'lev':pbelow})
    correction = -fbelow/2*(pbelow-pabove)*(b>data.lev[0])
    pabove -= (pbelow==pabove) 
    aboveintegral = (pbelow-b)*fabove+(fbelow-fabove)*(pbelow-pabove)*(1-((b-pabove)/(pbelow-pabove))**2)/2+correction
    layeraverage  = (belowintegral+innerintegral+aboveintegral)/(a-b)
    return layeraverage

In [ ]:
thetaeb  = get_layer_average(thetae,ps,pbltop)*np.sqrt(-1+2*(ps>lfttop))
thetael  = get_layer_average(thetae,pbltop,lfttop)
thetaels = get_layer_average(thetaes,pbltop,lfttop)

## Calculate $B_L$

The weighted contributions of the boundary layer ($w_B$) and lower free-troposphere ($w_L$) are calculated following [Adames et al. (2021)](https://journals.ametsoc.org/view/journals/atsc/78/2/jas-d-20-0074.1.xml) Equations 5a and 5b:

$$ w_B = \frac{\Delta p_B}{\Delta p_L} \ln{\left(1 + \frac{\Delta p_L}{\Delta p_B}\right)} $$

$$ w_L = 1 - w_B $$

In [ ]:
def calc_wb(ps,pbltop,lfttop):
    pblthickness = ps-pbltop
    lftthickness = pbltop-lfttop
    wb = (pblthickness/lftthickness)*np.log((pblthickness+lftthickness)/pblthickness)
    return wb

In [ ]:
wb = calc_wb(ps,pbltop,lfttop)
wl = 1-wb  

Following [Ahmed and Neelin (2021)](https://agupubs.onlinelibrary.wiley.com/doi/full/10.1029/2021GL094108) Equation 1, average buoyancy in the lower troposphere ($B_L$) is calculated as:

$$ B_L = \frac{g}{\overline{\kappa}_L\theta_{e0}}\left[w_B\underbrace{\left(\frac{\theta_{eB}-\theta^{*}_{eL}}{\theta^{*}_{eL}}\right)\theta_{e0}}_{\mathrm{CAPE_L}} - w_L\underbrace{\left(\frac{\theta^{*}_{eL}-\theta_{eL}}{\theta^{*}_{eL}}\right)\theta_{e0}}_{\mathrm{SUBSAT_L}}\right] $$

In [ ]:
thetae0 = 340. 
cape    = ((thetaeb-thetaels)/thetaels)*thetae0
subsat  = ((thetaels-thetael)/thetaels)*thetae0
bl      = (9.81/(3.*thetae0))*((wb*cape)-(wl*subsat))

## Save Dataset

Put all variables into a singular Xarray.Dataset, and save as a netCDF file to the user-defined save directory (```SAVEDIR```).

In [ ]:
def dataset(wb,wl,cape,subsat,bl,pr,author=AUTHOR,email=EMAIL):
    vardata = {'wb':([*wb.dims],wb.data),
               'wl':([*wl.dims],wl.data),
               'cape':([*cape.dims],cape.data),
               'subsat':([*subsat.dims],subsat.data),
               'bl':([*bl.dims],bl.data),
               'pr':([*pr.dims],pr.data)}
    coords  = {'time':bl.time.data,'lat':bl.lat.data,'lon':bl.lon.data}
    ds = xr.Dataset(vardata,coords)
    ds.wb.attrs     = dict(long_name='Fractional contribution of the boundary layer',units='0-1')
    ds.wl.attrs     = dict(long_name='Fractional contribution of the lower free-troposphere',units='0-1')
    ds.cape.attrs   = dict(long_name='Undilute plume buoyancy',units='K')
    ds.subsat.attrs = dict(long_name='Subsaturation in the lower-free troposphere',units='K')
    ds.bl.attrs     = dict(long_name='Average buoyancy in the lower troposphere',units='m/s²')
    ds.pr.attrs     = dict(long_name='Precipitation flux',units='mm/day')
    ds.time.attrs   = dict(long_name='Time')
    ds.lat.attrs    = dict(long_name='Latitude',units='$^\circ$N')
    ds.lon.attrs    = dict(long_name='Longitude',units='$^\circ$E')
    ds.attrs        = dict(description='Calculated following Ahmed & Neelin (2021) Eq. 1',
                           history=f'Created on {datetime.today().strftime("%Y-%m-%d")} by {author} ({email})')
    return ds

def save(ds,filename,savedir=SAVEDIR):
    filepath = f'{savedir}/{filename}'
    ds.to_netcdf(filepath)

In [ ]:
ds = dataset(wb,wl,cape,subsat,bl,pr)
save(ds,'OBS_bl_pr_POD.nc')